In [13]:
import pyspark
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from datetime import datetime
import pytz

In [14]:
account_name = "REDACTED"
account_key = "REDACTED"

spark = (
    SparkSession
        .builder
        .master('local[*]')
        .appName("Ingestion")
        .config("spark.driver.memory", "4g")
        .config("fs.azure.account.auth.type." + account_name + ".dfs.core.windows.net", "SharedKey")
        .config("fs.azure.account.key." + account_name + ".dfs.core.windows.net", account_key)
        .getOrCreate()
)

sc = spark.sparkContext

In [15]:
container_name = 'data'
path_to_table = '/training/'

def readDataframeFromAdls(spark_session, container_name, path_to_table, table_name):
    return (
        spark_session
            .read
            .parquet(
                f"abfss://{container_name}@REDACTED.dfs.core.windows.net{path_to_table}{table_name}"
            )
    ) 

In [16]:
#take approx. 255000 samples
df_env_indoor = (
    readDataframeFromAdls(spark, container_name, path_to_table, 'df_env_indoor')
        .withColumn('TABLE', F.lit('df_env_indoor'))
        .sample(0.633)
)
df_env_outdoor = (
    readDataframeFromAdls(spark, container_name, path_to_table, 'df_env_outdoor')
        .withColumn('TABLE', F.lit('df_env_outdoor'))
)
df_geo_labeled = (
    readDataframeFromAdls(spark, container_name, path_to_table, 'df_geo_labeled')
        .withColumn('TABLE', F.lit('df_geo_labeled'))
)
df_man_labeled = (
    readDataframeFromAdls(spark, container_name, path_to_table, 'df_man_labeled')
        .withColumn('TABLE', F.lit('df_man_labeled'))
)

In [17]:
df_env_indoor.groupBy('ENV_LABEL').count().orderBy('count').show()

+---------+------+
|ENV_LABEL| count|
+---------+------+
|        i|258209|
+---------+------+



In [18]:
df_env_outdoor.groupBy('ENV_LABEL').count().orderBy('count').show()

+---------+------+
|ENV_LABEL| count|
+---------+------+
|        o|257617|
+---------+------+



In [19]:
df_geo_labeled.groupBy('ENV_LABEL').count().orderBy('count').show()

+---------+------+
|ENV_LABEL| count|
+---------+------+
|        o|  6695|
|        i| 10077|
|        v|445493|
+---------+------+



In [20]:
df_man_labeled.groupBy('ENV_LABEL').count().orderBy('count').show()

+---------+-----+
|ENV_LABEL|count|
+---------+-----+
|    METRO|   35|
|      BUS|  391|
|    TRAIN|  845|
|      CAR| 1060|
|  OUTDOOR| 2034|
|   INDOOR|46387|
+---------+-----+



In [21]:
df_geo_labeled_io = (
    df_geo_labeled
        .filter(F.col('ENV_LABEL').isin(['i', 'o']))
)

#take approx. 5000 samples
df_geo_labeled_v = (
    df_geo_labeled
        .filter(F.col('ENV_LABEL') == 'v')
        .sample(0.009)
)

df_geo_labeled_union = df_geo_labeled_io.union(F.broadcast(df_geo_labeled_v))

In [22]:
# take approx. 6000 indoor samples 
df_man_labeled_i = (
    df_man_labeled
        .filter(F.col('ENV_LABEL').isin(['METRO', 'INDOOR']))
        .sample(0.093)
)

df_man_labeled_o = (
    df_man_labeled
        .filter(~F.col('ENV_LABEL').isin(['METRO', 'INDOOR']))
)

df_man_labeled_union = df_man_labeled_i.union(F.broadcast(df_man_labeled_o))

In [23]:
df_union = (
    df_env_indoor
        .union(F.broadcast(df_env_outdoor))
        .union(F.broadcast(df_geo_labeled_union))
        .union(F.broadcast(df_man_labeled_union))
        .withColumn('STIME', F.to_timestamp(F.col('STIME')))
        .withColumn('STIME_SEC', F.col('STIME').cast('long'))
)

In [24]:
@F.udf(returnType=IntegerType())
def epochToWeekday(epoch):
    return int(datetime.fromtimestamp(epoch).strftime('%w'))

In [25]:
@F.udf(returnType=IntegerType())
def epochToHour(epoch):
    return int(datetime.fromtimestamp(epoch).strftime('%-H'))

In [26]:
df_weekday = df_union.withColumn('WEEKDAY', epochToWeekday(F.col('STIME_SEC')))
df_hour = df_weekday.withColumn('HOUR', epochToHour(F.col('STIME_SEC')))

In [27]:
df_weekend = (
    df_hour
        .withColumn('WEEKEND', 
            F.when(
                F.col('WEEKDAY').isin([6, 0]), 1
            )
            .otherwise(0)
        )
    )

In [28]:
def isSummerTime(dt=datetime.now(), timezone="Europe/Berlin"):
    timezone = pytz.timezone(timezone)
    timezone_aware_date = timezone.localize(dt, is_dst=None)
    return timezone_aware_date.tzinfo._dst.seconds != 0

In [29]:
hours_added = 1
if isSummerTime():
    hours_added = 2

df_time_day = (
    df_weekend
        .withColumn('TIME_DAY', 
            F.when(
                (F.col('HOUR') + F.lit(hours_added)).between(6, 11), 'MORNING'
            )
            .otherwise(
                F.when(
                    (F.col('HOUR') + F.lit(hours_added)).between(12, 17), 'AFTERNOON'
                )
                .otherwise(
                    F.when(
                        (F.col('HOUR') + F.lit(hours_added)).between(18, 24), 'NIGHT'
                    )
                    .otherwise('NIGHT')
                )
            )
        )
    )

In [30]:
@F.udf(returnType=DoubleType())
def dbmToMw(level):
    if level is None:
        return None
    else:
        return 10**((level)/10.)

In [31]:
df_level = df_time_day.withColumn('LEVEL_MW', dbmToMw(F.col('LEVEL')))

In [32]:
df_label = (
    df_level
        .withColumn('ENV_LABEL', 
            F.when(F.col('ENV_LABEL').isin(['METRO', 'INDOOR', 'i']), 'i')
            .otherwise('o')
        )
    )

In [33]:
df_label.groupBy(['ENV_LABEL', 'TABLE']).count().orderBy('count').show()


+---------+--------------+------+
|ENV_LABEL|         TABLE| count|
+---------+--------------+------+
|        o|df_man_labeled|  4330|
|        i|df_man_labeled|  4389|
|        i|df_geo_labeled| 10077|
|        o|df_geo_labeled| 10679|
|        o|df_env_outdoor|257617|
|        i| df_env_indoor|258209|
+---------+--------------+------+



In [34]:
df_label.groupBy('ENV_LABEL').count().orderBy('count').show()

+---------+------+
|ENV_LABEL| count|
+---------+------+
|        o|272626|
|        i|272675|
+---------+------+



In [35]:
df_drop = df_label.drop('STIME', 'TABLE', 'STIME_SEC', 'WEEKDAY', 'HOUR')

In [36]:
df_select = df_drop.select(sorted(df_drop.columns))

In [37]:
df_select.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df_select.columns]).show()

+--------+---------------+--------------+-----------+--------+---------+---------+------------+---------+---------+-------------+----------+----------+-----+--------+------+--------+--------+--------+---------+------+------+-----+-----+--------+-----+-------+--------------+
|ACCURACY|ACTIVITY_CONFID|ACTIVITY_STATE|BATT_CHARGE|DISTANCE|ENV_LABEL|GPS_DELAY|LAG_ACTIVITY|LAG_LTIME|LAG_SPEED|LEAD_ACTIVITY|LEAD_LTIME|LEAD_SPEED|LEVEL|LEVEL_MW| LIGHT|MAGNET_X|MAGNET_Y|MAGNET_Z|PROXIMITY|  QUAL|SOURCE|SPEED|STYPE|TIME_DAY| TYPE|WEEKEND|WIFI_CONNECTED|
+--------+---------------+--------------+-----------+--------+---------+---------+------------+---------+---------+-------------+----------+----------+-----+--------+------+--------+--------+--------+---------+------+------+-----+-----+--------+-----+-------+--------------+
|      19|              0|             0|        253|  217646|        0|        0|       35280|   217646|   217646|        35395|    218406|    218406|65518|   65518|332149|  

In [25]:
container_name = 'data'
path_to_table = '/training/'

def writeDataframeToAdls(dataframe, container_name, path_to_table, table_name, mode='overwrite'):
    (dataframe
        .write
        .mode(mode)
        .format("table_name")
        .parquet(f"abfss://{container_name}@REDACTED.dfs.core.windows.net{path_to_table}{table_name}")
    )

In [26]:
writeDataframeToAdls(df_select, container_name, path_to_table, "df_training")